# __Welcome to the MMS Data Engineering Challenge!__

This notebook presents a series of problems for you to solve. Note that some problems do not have a single optimal solution, but rather multiple good solutions. We are interested in seeing your approach to these problems. You have considerable flexibility in how you address them. As a Data Engineer, we value your versatility, so solutions may include Python scripts, SQL queries, plain text explanations, or even diagrams.

Some exercises are much simpler than others. If you get stuck, feel free to skip the problem and move on. __We will also accept partially completed challenges.__ We want to understand your thought process and the tools you are familiar with. If you don't know how to code a solution but understand how to solve it conceptually, use plain text to explain your answer.

In the container folder you will also find the file `sql_definitions.sql` containing all sql table definitions shown bellow. And the original image of the tables in exercises 3 and 4: `duplicates.png` and `employees.png`.

**DISCLAIMER**: This Data Engineering Challenge is disguised as a Jupyter Notebook, but don't let that fool you—it's only relevant for the first exercise. For the remaining exercises, feel free to use the code cells or convert them to Markdown cells as you prefer.

## Guillermo S. Sanjuan Ortiz

## Exercise 1

Write a Python function `zeros_and_ones(n, o)` to print every possible combination of n - o zeros and o ones, for some given n and o.
_Note that you have to create at least the mentioned function, but you can create others if needed._

__Input__: The input consists of two natural numbers n and o, such that n > 0 and 0 ≤ o ≤ n.

__Output__: Print all the combinations of n − o zeros and o ones, one per line and in lexicographical order.

__Example__

```
> zeros_and_ones(5, 2)

0 0 0 1 1
0 0 1 0 1
0 0 1 1 0
0 1 0 0 1
0 1 0 1 0
0 1 1 0 0
1 0 0 0 1
1 0 0 1 0
1 0 1 0 0
1 1 0 0 0
```

The solution is preferred in Python, but in case you don't feel confident enough using Python, provide the solution in another programming language of your choice.

In [169]:
def zeros_and_ones(n:int, o:int)->None:
  def print_track(n:int, cars_position:list)->None:
    """
    Prints the track, based on the position of cars
    """
    track = ['0'] * n
    for i in cars_position:
      track[i] = '1'
    print(' '.join(track))
    return
  def race_continue(n:int, cars_position:list)->list[int]|None:
    """
    Given current position of the cars, calculates the next move and returns next
    position of the cars. If race ends, returns None.
    """
    new_cars_position = cars_position.copy()
    for i in range(len(cars_position) -1, 0, -1): # Loop over all, but participant one
      if cars_position[i] - cars_position[i-1]  > 1: # If they have space ahead move
            new_cars_position[i] -= 1
            return new_cars_position
    if new_cars_position[0] == 0: # If parcitipant one is at the line and no one moves
      return # then race ends
    else: # If no one moves, means they are all packed so:
      new_cars_position[0] -= 1 # Participant one moves
      for j in range(1, len(new_cars_position)): # The rest return to initial step
          new_cars_position[j] = n - (len(new_cars_position) - j)
    return new_cars_position
  if n < 0 or o < 0 or o > n: # If input is invalid, raise error
    raise ValueError("Invalid input")
  if o == 0: # Edge case, no racers
    print(' '.join(['0']*n))
    return
  # Initial cars position
  cars_position = list(range(n - o, n))
  while cars_position: # Loop continues until cars_position is null
    print_track(n, cars_position)
    cars_position = race_continue(n, cars_position)

This is just to run your code:

In [175]:
zeros_and_ones(5, 2)

0 0 0 1 1
0 0 1 0 1
0 0 1 1 0
0 1 0 0 1
0 1 0 1 0
0 1 1 0 0
1 0 0 0 1
1 0 0 1 0
1 0 1 0 0
1 1 0 0 0


## Comments:

I thought of it like a race of n steps and o participants, where the first one starts in oth position, the second one in o + 1th position, and so on.

They move as soon as they have space ahead, but only if the previous racer have moved, so I loop in inverse, starting with last participant.

I don't loop over car 1 because he can always move, but if no one moves, then participant one moves and the others return to the starting point, then the loop begins again until no one has moved and participant is on the line (pos 0).

In my implementation, I focus on participants, so I made a `print_track` helper function to print the track in every move.

I also created a function `race_continue` to calculate next move, that returns the next step of the cars.

This exercise was the hardest one in my opinion.



## Exercise 2

How can the following SQL statement be re-written more easy?

```
SELECT product_id, IF(has_sales=TRUE, TRUE, FALSE) as purchase_flag,
FROM salesdata
```

### Response
```
SELECT
  product_id,
  has_sales AS purchase_flag
FROM
  salesdata
```

## Exercise 3

There are several ways to find duplicated rows in a table - write a SQL statement of your choice to find duplicates in table C.

Table C
<img src="duplicates.png" width="200px"/>

## Response
```
SELECT
  *
FROM
  outlets_info
GROUP BY
  outlet_name,
  country,
  region,
  address,
  zipcode
HAVING
  COUNT(*) > 1
```

## Exercise 4
EmployeeID, Department, name, salary

1. Write a SQL query to get the third-highest salary of an employee from `employee_table`.
2. Write a SQL query to calculate the percentage (%) that each Employee's salary contributes to their respective Department's total salary.

employee_table
<img src="employees.png" width="300px"/>

## Response
### 4.1
```
SELECT
  DISTINCT salary
FROM
  employee_table
ORDER BY
  salary DESC
LIMIT 1 OFFSET 2
```
### 4.2
```
SELECT
  EmployeeID,
  Department,
  name,
  salary,
  (salary / (SELECT SUM(salary) FROM employee_table WHERE Department = e.Department)) * 100 AS salary_percentage
FROM
  employee_table e
ORDER BY
  salary_percentage DESC
  Department DESC
```

# Exercise 5

The following sales_daily table is widely used within MMS for multiple analytics.

```
create table sales_daily
(
    country          STRING,
    sales_date       DATE,
    outlet_id        INT64,
    product_category INT64,
    product_id       INT64,
    brand            STRING,
    sales_qty        INT64,
    sales_value      NUMERIC
);
```

One of our primary use cases within our domain is to analyze the daily sales trends across different countries. However, due to the high volume of products sold each day, these queries consume substantial memory and processing time.
- Write a query to retrieve the daily sales evolution per country.
- Suppose this query takes too long to run. Explain how we could optimize the previous definition of the table `sales_daily` in order to obtain a better performance in this query.
        -> If you think it is needed, do not hesitate to also provide the code for doing so.
        -> Taking into account that this table is hosted in BigQuery, adapt the explanation (as much as your can) using BigQuery terminology instead of generic SQL terminology.  

## Response
The query for retrieving the evolution could be:
```
SELECT
  country,
  sales_date,
  SUM(sales_qty) AS total_sales_qty,
  SUM(sales_value) AS total_sales_value
FROM
  sales_daily
GROUP BY
  country,
  sales_date
ORDER BY
  country,
  sales_date;
```
This query assumes that sales_value correspond to the value for the whole quantity of items sold, and not to the value of one unit.

### Optimization explained

However, this approach is very expensive in computation, since it processes all the table each time the query is run. In order to avoid this, we should follow the partitioning strategy, alongside with a kpi persistent table:

First we modify the table creation query to partition it by date:

```
CREATE TABLE sales_daily
PARTITION BY sales_date
(
    country          STRING,
    sales_date       DATE,
    outlet_id        INT64,
    product_category INT64,
    product_id       INT64,
    brand            STRING,
    sales_qty        INT64,
    sales_value      NUMERIC
);
```
When a table is partitioned in BigQuery and a filter is applied on the query (this can be enforced), we will only be charged for the rows in the partitions being queried.

The next step would be creating a table where the daily summary would be inserted:

```
CREATE TABLE sales_by_country_daily
(
  country            STRING,
  sales_date         DATE,
  total_sales_qty    INT64,
  total_sales_value  NUMERIC
);
```

Finally, the next step would be to create a job that daily runs the query above, but filtering by a particular date:

```
MERGE INTO sales_by_country_daily AS target
USING (
  SELECT
    country,
    sales_date,
    SUM(sales_qty) AS total_sales_qty,
    SUM(sales_value) AS total_sales_value
  FROM
    sales_daily
  WHERE
    sales_date = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  GROUP BY
    country,
    sales_date
) AS source
ON target.country = source.country AND target.sales_date = source.sales_date
WHEN MATCHED THEN
  UPDATE SET
    total_sales_qty = source.total_sales_qty,
    total_sales_value = source.total_sales_value
WHEN NOT MATCHED THEN
  INSERT (country, sales_date, total_sales_qty, total_sales_value)
  VALUES (source.country, source.sales_date, source.total_sales_qty, source.total_sales_value);
```

The query above bears in mind that maybe a daily job needs to be rerunned and handles it by updating a row when already in the summary table to avoid duplicates.

Note that in the explanation above, I make two assumptions:

- Past data is doesn't change.
- The table isn't big enough to hit the number of partitions quota.

In case past data could change, which is a highly likely scenario since there can occur issues like latency when uploading data to `sales_daily`, there could be two different strategies:

  - Simple strategy: Stablishing a hard limit to past data to change based on most likely scenario. If we know that 100% of these cases occurs with data of the past 10 days, we could run each day a query that calculates the summary of the past 10 days instead of only the last day.
  - Complex but more cost efficient strategy: Building a system that re-runs the query for a particular date when data is inserted/updated for that date (only if this date is a past one).

Regarding the hitting quota scenario, we could stablish a sharding strategy, where we have a table for each year for example. Another strategy would be to partition  by week or month instead of day, but it would incur in addional cost per query so I would discard last option.


The last thing I want to mention is clustering. By clustering by country this would increase the speed of the query but won't improve the cost:
```
CREATE TABLE sales_daily
PARTITION BY sales_date
CLUSTER BY country
(
  country          STRING,
  sales_date       DATE,
  outlet_id        INT64,
  product_category INT64,
  product_id       INT64,
  brand            STRING,
  sales_qty        INT64,
  sales_value      NUMERIC
);
```

This is due to the fact that BigQuery can take advantage of pre-sorted data within each partition, making the `GROUP BY` more efficient. It won't improve the cost as we always run the query for all countries.



# Exercise 6

Suppose we have the following tables in a BigQuery.

**sales_daily**  
Same definition as in previous exercise.


**curr_exchange**  
This table provides us the exchange rate from the country's local currency to EUR.  
Note that the Sales value in `sales_daily` is in local currency. The table has the following structure:
```
create table curr_exchange
(
    country       STRING,
    ex_loc_to_eur NUMERIC
);
```


**outlets_info**  
This table contains relevant information of the outlets and has the following structure:
```
create table outlets_info
(
    outlet_id   INT64,
    outlet_name STRING,
    country     STRING,
    region      STRING,
    address     STRING,
    zipcode     STRING,
);
```


**output_table**  
This table is empty and needs to be filled. `sales_value_eur` is the conversion of `sales_value` to EUR.
```
create table output_table
(
    country         STRING,
    sales_date      DATE,
    outlet_id       INT64,
    outlet_name     STRING,
    region          STRING,
    sales_value     NUMERIC,
    sales_value_eur NUMERIC
);
```


## Part 1
Create a query to fill `output_table`. Join the tables accordingly to create the desired output.


## Part 2
A new table definition has been added:
```
create table products_info
(
    product_id   INT64,
    product_name STRING,
    is_own_brand BOOLEAN
);
```
This table shows which products have an Own Brand of MediaMarkt (i.e. `OK` or `ISY` are an Own Brand of MediaMarkt).  
Modify the query in order to exclude all those products which are Own Brands.


## Part 3
There is now a new version of the currency exchange table which includes date. As you may know, the currency exchange rates is highly variable. For obtaining the right sales value in eur, we need to use the appropiate date's currency exchange rate. Modify the query to use the new version of the exchange rate table `curr_exchange_v2`.
```
create table curr_exchange_v2
(
    country       STRING,
    rate_date     DATE,
    ex_loc_to_eur NUMERIC
);
```

## Part 4
Provide a schema / diagram to illustrate how the tables are combined to produce the `output_table`.


## Part 5
This query needs to run daily and update `output_table` accordingly. Explain how you would set this up to run every day in GCP. Imagine that `output_table` and the rest of the tables are available in our BigQuery.

_Note that: We discard creating a view instead of a table because the response time is too slow due to the size of the sources._  
_Note that2: For the scope of this exercise we do not consider the option of creating a materialized view, but rather we want a table that gets updated every day with the results of our previously implemented query._


## Part 6
As you have seen, the query for filling `output_table` gets constant modifications. The sources we use are in constant re-definitions and improvements. For this reason, we need to adapt and modify this query often. We want to do these modifications under a version controlled (git) environment. Explain how we can set this up.  
Ideally:
1. Have the SQL code of our query, version controlled in GitHub.
2. Once a change is pushed into the master branch, the query needs to be modified in GCP accordingly.
3. The next daily run must occurs with the new version of the code.

Explain how to implement this.


## ANSWER

## Part 1

```
INSERT INTO output_table (
    country,
    sales_date,
    outlet_id,
    outlet_name,
    region,
    sales_value,
    sales_value_eur
)
SELECT
    sales_daily.country,
    sales_daily.sales_date,
    sales_daily.outlet_id,
    outlets_info.outlet_name,
    outlets_info.region,
    SUM(sales_daily.sales_value) AS sales_value,
    SUM(sales_daily.sales_value * curr_exchange.ex_loc_to_eur) AS sales_value_eur
FROM
    `exercice6.sales_daily` sales_daily
JOIN
    `exercice6.outlets_info` outlets_info ON sales_daily.outlet_id = outlets_info.outlet_id
JOIN
    `exercice6.curr_exchange` curr_exchange ON sales_daily.country = curr_exchange.country
GROUP BY sales_date, outlet_id, country, outlet_name, region
```

## Part 2

```
INSERT INTO output_table (
    country,
    sales_date,
    outlet_id,
    outlet_name,
    region,
    sales_value,
    sales_value_eur
)
SELECT
    sales_daily.country,
    sales_daily.sales_date,
    sales_daily.outlet_id,
    outlets_info.outlet_name,
    outlets_info.region,
    sales_daily.sales_value,
    sales_daily.sales_value * curr_exchange.ex_loc_to_eur AS sales_value_eur
FROM
    sales_daily
JOIN
    outlets_info ON sales_daily.outlet_id = outlets_info.outlet_id
JOIN
    curr_exchange ON sales_daily.country = curr_exchange.country
JOIN
    products_info ON sales_daily.product_id = products_info.product_id
WHERE
    products_info.is_own_brand = FALSE;
```

## Part 3
```
INSERT INTO output_table (
    country,
    sales_date,
    outlet_id,
    outlet_name,
    region,
    sales_value,
    sales_value_eur
)
SELECT
    sales_daily.country,
    sales_daily.sales_date,
    sales_daily.outlet_id,
    outlets_info.outlet_name,
    outlets_info.region,
    SUM(sales_daily.sales_value) AS sales_value,
    SUM(sales_daily.sales_value * curr_exchange.ex_loc_to_eur) AS sales_value_eur
FROM
    `exercice6.sales_daily` sales_daily
JOIN
    `exercice6.outlets_info` outlets_info ON sales_daily.outlet_id = outlets_info.outlet_id
JOIN
    `exercice6.curr_exchange_v2` curr_exchange ON sales_daily.country = curr_exchange.country
    AND sales_daily.sales_date = curr_exchange.rate_date
JOIN
    `exercice6.products_info` products_info ON sales_daily.product_id = products_info.product_id
WHERE
    products_info.is_own_brand is FALSE
GROUP BY sales_date, outlet_id, country, outlet_name, region
```

## Part 4

schema
<img src="https://drive.google.com/file/d/11rZkX3b8yob2ETvwhEpDJkIHZwBIQ1DV/view?usp=sharing" width="300px"/>


## Part 5

From my last interview with Sebastian I learned that Dataform is one of the main tools that you use so I will choose Dataform to deploy this job.

First step would be defining the model of dataform `output_table.sqlx`:

```
config {
  type: "incremental",
  name: "output_table",
  schema: "exercice6",
  uniqueKey: ["sales_date", "outlet_id"],
  tags: ["daily_job"]
}
SELECT
    sales_daily.country,
    sales_daily.sales_date,
    sales_daily.outlet_id,
    outlets_info.outlet_name,
    outlets_info.region,
    SUM(sales_daily.sales_value) AS sales_value,
    SUM(sales_daily.sales_value * curr_exchange.ex_loc_to_eur) AS sales_value_eur
FROM
    `exercice6.sales_daily` sales_daily
JOIN
    `exercice6.outlets_info` outlets_info ON sales_daily.outlet_id = outlets_info.outlet_id
JOIN
    `exercice6.curr_exchange_v2` curr_exchange ON sales_daily.country = curr_exchange.country
    AND sales_daily.sales_date = curr_exchange.rate_date
JOIN
    `exercice6.products_info` products_info ON sales_daily.product_id = products_info.product_id
WHERE
    products_info.is_own_brand is FALSE
GROUP BY sales_date, outlet_id, country, outlet_name, region

```

Once I have the model, I can deploy it into dataform.

## Part 6 CI/CD

For this exercise I actually created a real CI/CD pipeline and put it in practice. I invite you to explore the [public repo](https://github.com/Guillethecoder/media_markt_dataform) I created for this.

I will now describe the steps I've taken:

- Created a GCP project for easy removal of resources once there is no need to keep it alive.
- Created a BQ dataset and all raw tables, seeded tables with data generated with AI.
- Created a new Dataform repository.
- Connected it to my github repo, which required creating a fine-grained password in github and storing it in Secret Manager. Then giving the Dataform SA access to Secret Manager. Finally completing the setup.
- Since this is my first time with Dataform for a long time (I've been working with DBT which is pretty similar) I created a Development Workspace to play around, mostly to make sure I understood how to setup the package.json and the dataform.json.
- After that, I started working directly in the repo to simulate a real environment. I used GitFlow with main and dev branches, alongside with the feature ones, so I invite you to take a look at the commits and PRs.
- Finally I executed the workflow, at first it fails because of the permissions of the SA I used, but after fixing this issue it worked fine and results were as expected 😀.

I can show you the GCP environment in our next interview. Glad to say this part of the challenge was the most fun.